# Text Generation: How to run inference on the endpoint you have created?

In [1]:
import json
import boto3

### Query endpoint that you have created

In [2]:
endpoint_name = "jumpstart-dft-hf-llmneuron-mistral-20240417-224535"


def query_endpoint(payload):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload).encode("utf-8")
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

### Supported parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments.
***

In [3]:
from typing import Dict, List


def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)


In [4]:
instructions = [{"role": "user", "content": "what is the recipe of mayonnaise?"}]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256}
}
response = query_endpoint(payload)
print(response)

[{'generated_text': '<s>[INST] what is the recipe of mayonnaise? [/INST] 1 cup mayonnaise is made by combining 1/2 cup of vegetable oil, 1/2 cup of egg yolks, 1 tablespoon of lemon juice, 1 tablespoon of Dijon mustard, 1 tablespoon of honey, and a pinch of salt and pepper.\n\nTo make the mayonnaise, whisk together the egg yolks and lemon juice in a small bowl until they are light and frothy. Slowly drizzle in the vegetable oil, whisking constantly, until the mixture thickens and emulsifies. Stir in the Dijon mustard, honey, salt, and pepper to taste.\n\nMayonnaise can be stored in an airtight container in the refrigerator for up to a week. To make it fresher, you can add a tablespoon of fresh lemon juice or white wine vinegar before storing it.</s>'}]


In [5]:
instructions = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256}
}
response = query_endpoint(payload)
print(response)

[{'generated_text': "<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 1. The Eiffe

In [6]:
instructions = [
    {
        "role": "user",
        "content": "In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month?",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256}
}
response = query_endpoint(payload)
print(response)

[{'generated_text': '<s>[INST] In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month? [/INST] 1. `file1.txt`\n2. `file2.txt`\n3. `file3.txt`\n4. `file4.txt`\n5. `file5.txt`\n6. `file6.txt`\n7. `file7.txt`\n8. `file8.txt`\n9. `file9.txt`\n10. `file10.txt`\n\nNote: The above list is just an example and may not be accurate. It is recommended to use the `find` command with appropriate options to get the desired output.</s>'}]


In [7]:
instructions = [
    {
        "role": "user",
        "content": "What is the difference between inorder and preorder traversal? Give an example in Python.",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256}
}
response = query_endpoint(payload)
print(response)

[{'generated_text': '<s>[INST] What is the difference between inorder and preorder traversal? Give an example in Python. [/INST] 1. Inorder traversal is a way of traversing a binary tree where the left subtree is traversed first, followed by the root node, and then the right subtree. This is done in a left-root-right order.\n\nExample in Python:\n```python\ndef inorder_traversal(root):\n    if root:\n        inorder_traversal(root.left)\n        print(root.val)\n        inorder_traversal(root.right)\n\n# Example usage\nroot = Node(5)\nroot.left = Node(3)\nroot.right = Node(7)\ninorder_traversal(root)  # Output: 3 5 7\n```\n\n2. Preorder traversal is a way of traversing a binary tree where the root node is visited first, followed by the left subtree and then the right subtree. This is done in a root-left-right order.\n\nExample in Python:\n```python\ndef preorder_traversal(root):\n    if root:\n        print(root.val)\n        preorder_traversal('}]


In [9]:
instructions = [
    {
        "role": "user",
        "content": "What is the difference between inorder and preorder traversal? Give an example in Python.",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 11}
}
response = query_endpoint(payload)
print(response)

[{'generated_text': '<s>[INST] What is the difference between inorder and preorder traversal? Give an example in Python. [/INST] 1. Inorder traversal: Inorder traversal'}]
